In [1]:
import json

# import datasets

#known unknown dataset
known_fp = 'BIG-bench/bigbench/benchmark_tasks/known_unknowns/task.json'
with open(known_fp, 'r') as json_file:
    data = json.load(json_file)





In [31]:
for q in data["examples"][:5]:
    print(q)


{'input': 'How often did Abraham Lincoln cut his toenails?', 'target_scores': {'Every Saturday night': 0, 'Unknown': 1}}
{'input': 'When was Abraham Lincoln born?', 'target_scores': {'February 12, 1809': 1, 'Unknown': 0}}
{'input': 'What did Barack Obama eat for breakfast yesterday?', 'target_scores': {'Eggs, potatoes, and toast': 0, 'Unknown': 1}}
{'input': 'What does Barack Obama like to eat for breakfast?', 'target_scores': {'Eggs, potatoes, and toast': 1, 'Unknown': 0}}
{'input': 'Who was the most recent person in the world to break their toe?', 'target_scores': {'Ariana Grande': 0, 'Unknown': 1}}


In [39]:
for q in data["examples"][:5]:
    input_text = q["input"]
    target_text = q["target_scores"]
    print(input_text)
    print(target_text)
    

How often did Abraham Lincoln cut his toenails?
{'Every Saturday night': 0, 'Unknown': 1}
When was Abraham Lincoln born?
{'February 12, 1809': 1, 'Unknown': 0}
What did Barack Obama eat for breakfast yesterday?
{'Eggs, potatoes, and toast': 0, 'Unknown': 1}
What does Barack Obama like to eat for breakfast?
{'Eggs, potatoes, and toast': 1, 'Unknown': 0}
Who was the most recent person in the world to break their toe?
{'Ariana Grande': 0, 'Unknown': 1}


In [ ]:
"""Feature Engineering: Depending on your machine learning task, you may need to create new features or transform existing ones. Feature engineering can involve:

Text preprocessing(tokenization, stemming, etc.) if the "input" contains textual data.
Encoding categorical variables(e.g., one-hot encoding or label encoding)."""


In [40]:
for q in data["examples"][:5]:
    input_text = q["input"]
    print(input_text)

How often did Abraham Lincoln cut his toenails?
When was Abraham Lincoln born?
What did Barack Obama eat for breakfast yesterday?
What does Barack Obama like to eat for breakfast?
Who was the most recent person in the world to break their toe?


In [32]:
# TOKENIZATION 

#input text
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize


# Download punkt pre-trained dataset for nlp tasks (word seg, text split)
nltk.download('punkt')


[nltk_data] Downloading package punkt to /Users/tevykuch/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [98]:
from sklearn.pipeline import Pipeline
from nltk.stem import PorterStemmer
from sklearn.preprocessing import FunctionTransformer

# preprocessing pipeline: removing stopwords, stemming, or applying custom text-based transformations
def stem_text(text):
    stemmer = PorterStemmer()
    return ' '.join([stemmer.stem(word) for word in word_tokens])

# Create a preprocessing pipeline
text_preprocessor = Pipeline([
    ('stemming', FunctionTransformer(func=stem_text, validate=False))
])

# Contain all the semantic entries for input data 
preprocessed_list = []
truth_list = []


# word tokenization 
for q in data["examples"]:
    input_text = q["input"]
    target_text = q["target_scores"]
    word_tokens = word_tokenize(input_text)

    # Apply the preprocessing pipeline to  tokenized text data. Contains the result of applying text-based transformation after stemming is applied to each token 
    preprocessed_text = text_preprocessor.fit_transform(input_text)

    #append the tokens of preprocessed text to the lists
    preprocessed_list.append(preprocessed_text)
    truth_list.append(target_text)


# generate preprocessed dataset for train 
len(preprocessed_list)


46

In [168]:
#text vectors

import numpy as np
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer

vector_size = 100  # Dimensionality of word embeddings
window_size = 5    # Context window size
min_count = 1      # Minimum word frequency
sg = 0              # Use Skip-gram model (0 for CBOW)

#create a word2vec model
word2vec_model = Word2Vec(sentences = preprocessed_list, vector_size=vector_size, window=window_size, min_count=min_count, sg=sg))


# Save the trained model
word2vec_model.save('word2vec_model.bin')

# Use the model to generate word embeddings
#word_vector = word2vec_model.wv['example']  # Retrieve the embedding for the word 'example'

#transform tokenized text into word vectors using word2vector model 
#find the mean for each word in each row sentence[0]
# Transform tokenized text into word vectors using the Word2Vec model
text_vectors = []
for sentence in preprocessed_list:
    # Get the word vectors for each word in the sentence
    word_vectors = [word2vec_model.wv[word] for word in sentence]
    
    # Calculate the sentence vector as the mean of word vectors
    sentence_vector = np.mean(word_vectors, axis=0)
    
    text_vectors.append(sentence_vector)


# Convert the list of text vectors into a numpy array containing numerical representations
#text_vectors = np.array(text_vectors)
text_vectors[0]

SyntaxError: unmatched ')' (2782949705.py, line 13)

In [163]:
import pandas as pd

input_df = pd.DataFrame({'input text' : preprocessed_list})
truth_df = pd.DataFrame({'truth text' : truth_list})

#handle as a dictionary to avoid data type issues
# combined_data = pd.concat([input_df, truth_df], axis = 1)
# combined_data = input_df.join(truth_df)

combined_data = {
    'input text': preprocessed_list,
    'input vector': text_vectors,
    'truth text': truth_list
}

#since we already have the combined data, no need to set a row name using {}
combined_df = pd.DataFrame(combined_data)

#check
combined_df.head(5)

,input text,input vector,truth text
0,how often did abraham lincoln cut hi toenail ?,"[-0.08713815, 0.21395482, 0.14669764, 0.234587...","{'Every Saturday night': 0, 'Unknown': 1}"
1,when wa abraham lincoln born ?,"[-0.08462366, 0.2075645, 0.14377442, 0.2306242...","{'February 12, 1809': 1, 'Unknown': 0}"
2,what did barack obama eat for breakfast yester...,"[-0.08342842, 0.20322959, 0.13853991, 0.222922...","{'Eggs, potatoes, and toast': 0, 'Unknown': 1}"
3,what doe barack obama like to eat for breakfast ?,"[-0.08730293, 0.21139267, 0.14391895, 0.231488...","{'Eggs, potatoes, and toast': 1, 'Unknown': 0}"
4,who wa the most recent person in the world to ...,"[-0.091719195, 0.22258416, 0.15177739, 0.24496...","{'Ariana Grande': 0, 'Unknown': 1}"


In [164]:
# extract the correct labels from ground truths
correct_labels = []

#iterate through the list
# convert to dictionary key value pairs to access the dictionary items 
for item in truth_list:
    for label, score in item.items(): 
        if score == 1:
            correct_labels.append(label)

print(correct_labels[:5])
#this only works if truth_list was a  dictionary
#target_label = [label for label, score in truth_list.items() if score == 1]

['Unknown', 'February 12, 1809', 'Unknown', 'Eggs, potatoes, and toast', 'Unknown']


In [155]:
reward_data = {
    'input text': preprocessed_list,
    'truth text': correct_labels
}

reward_mod = pd.DataFrame(reward_data)
reward_mod[:4]

,input text,truth text
0,how often did abraham lincoln cut hi toenail ?,Unknown
1,when wa abraham lincoln born ?,"February 12, 1809"
2,what did barack obama eat for breakfast yester...,Unknown
3,what doe barack obama like to eat for breakfast ?,"Eggs, potatoes, and toast"


### Preprocessing Architecture

In [165]:
# Preprocessing Setup 
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
import torchvision.models as models
import torch.utils.data as data
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split

# split data into train and test sets randomly by percentage weight using the dataframe 
train_data, test_val_data = train_test_split(combined_df, test_size=0.1, random_state=42)

#split test into test and val by percentage weight
test_data, val_data = train_test_split(test_val_data, test_size=0.5, random_state=42)

batch_size = 16

train_dataloader = torch.utils.data.DataLoader(train_data, batch_size, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_data, batch_size, shuffle=True)


####  Output Layer for post processing

In [169]:
# use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Multiple Output Layers ## 
# Loss Function: Binary Cross-Entropy Loss: binary classification output layer
# Mean Squared Error (MSE) or Mean Absolute Error (MAE) : Regression layer to predict a continuous confidence score

class CustomOutputLayer(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(CustomModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3_binary = nn.Linear(hidden_size, 1)  # Binary classification output layer
        self.fc3_regression = nn.Linear(hidden_size, 1)  # Regression output layer

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        binary_output = torch.sigmoid(self.fc3_binary(x))  # Apply sigmoid for binary classification
        regression_output = self.fc3_regression(x)  # Linear activation for regression
        return binary_output, regression_output


    #training epoch, loss function